In [1]:
import os
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from glob import glob
from torchmetrics import Accuracy
from torchvision import models

In [2]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
ALL_CHAR_SET = NUMBER + ALPHABET
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 4

In [3]:
class OCRDataset(Dataset):
    def __init__(self, image_files, transform=None):
        self.image_files = image_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def encode(self, char):
        onehot = [0] * ALL_CHAR_SET_LEN
        idx = ALL_CHAR_SET.index(char)
        onehot[idx] += 1
        return onehot

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        img = Image.open(image_file)
        img = img.convert('L')
        label = image_file.split(os.sep)[-1][:-4]
        label_oh = []
        for i in label[:MAX_CAPTCHA]:  
            label_oh += self.encode(i)
        
        label_oh += [0] * (ALL_CHAR_SET_LEN * (MAX_CAPTCHA - len(label)))
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
        

In [4]:
class OCRDataModule(pl.LightningDataModule):
    
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size

        self.transform = transforms.Compose([
            transforms.Resize([64, 128]),
            transforms.ToTensor(),
        ])

    def setup(self, stage=None):
        train_image_files = glob('/kaggle/input/datasets/adritarahman3/ocr-dataset-1/OCR/Train/*.png')
        test_image_files = glob('/kaggle/input/datasets/adritarahman3/ocr-dataset-1/OCR/Test/*.png')

        self.train_dataset = OCRDataset(
            train_image_files,
            transform=self.transform
        )

        self.val_dataset = OCRDataset(
            test_image_files,
            transform=self.transform
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
        )




In [5]:
data_module = OCRDataModule(
    batch_size=32
)

data_module.setup()


In [6]:

class OCRModel(pl.LightningModule):
    def __init__(self, num_chars=ALL_CHAR_SET_LEN, max_length=MAX_CAPTCHA):
        super(OCRModel, self).__init__()
        self.save_hyperparameters()
        self.char_set = ALL_CHAR_SET

        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_chars * max_length)
        )

        self.train_accuracy = Accuracy(task='multiclass', num_classes=num_chars)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=num_chars)

        self.inference_transform = transforms.Compose([
            transforms.Resize([64, 128]),
            transforms.ToTensor(),
        ])

    def forward(self, x):
        out = self.resnet(x)
        return out.view(-1, self.hparams.max_length, self.hparams.num_chars)

    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)
        
        batch_size = x.size(0)
        y_hat = y_hat.view(batch_size * self.hparams.max_length, self.hparams.num_chars)
        y = y.view(batch_size, self.hparams.max_length, self.hparams.num_chars)
        y = y.argmax(dim=-1).view(-1)  

        loss = F.cross_entropy(y_hat, y)

        self.train_accuracy(y_hat.softmax(dim=-1), y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', self.train_accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)

        batch_size = x.size(0)
        y_hat = y_hat.view(batch_size * self.hparams.max_length, self.hparams.num_chars)
        y = y.view(batch_size, self.hparams.max_length, self.hparams.num_chars)
        y = y.argmax(dim=-1).view(-1)  

        loss = F.cross_entropy(y_hat, y)

        self.val_accuracy(y_hat.softmax(dim=-1), y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.val_accuracy, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler
        }

    def predict_image(self, image_path):
        
        img = Image.open(image_path).convert('L')
        img = self.inference_transform(img)
        img = img.unsqueeze(0) 

        img = img.to(self.device)
        
        self.eval()
        with torch.no_grad():
            output = self(img)  
            output = output.softmax(dim=-1)  
            pred_indices = output.argmax(dim=-1)  

        pred_text = ''.join(self.char_set[idx] for idx in pred_indices[0])
        return pred_text
      
        

In [7]:
model = OCRModel()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s] 


In [8]:
trainer = pl.Trainer(
        max_epochs=50,
)
trainer.fit(model, data_module)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
2026-02-18 03:42:49.248180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771386169.447783      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771386169.510781      55 cuda_blas.cc

┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ resnet         │ ResNet             │ 11.5 M │ train │     0 │
│ 1 │ train_accuracy │ MulticlassAccuracy │      0 │ train │     0 │
│ 2 │ val_accuracy   │ MulticlassAccuracy │      0 │ train │     0 │
└───┴────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 11.5 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 11.5 M                                                                                               
Total estimated model params size (MB): 46                                                                         
Modules in train mode: 74                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 
'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` 
from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., 
batch_size=batch_size)`.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` 
from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., 
batch_size=batch_size)`.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 
'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:317: The number of training batches 
(11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if
you want to see logs for the training epoch.

`Trainer.fit` stopped: `max_epochs=50` reached.


In [9]:
image_file = '/kaggle/input/datasets/adritarahman3/ocr-dataset-1/OCR/Test/1odm.png'
pred_text = model.predict_image(image_file)
print(pred_text)

1odm


In [10]:
pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0rc2
    Uninstalling packaging-26.0r

In [11]:
BATCH_SIZE=32
MAX_EPOCHS=50
ALL_CHAR_SET_LEN=36
MAX_CAPTCHA=4

In [12]:
import os
import shutil
import mlflow
import mlflow.pytorch
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import MLFlowLogger
from IPython.display import FileLink


MLFLOW_DIR = "/kaggle/working/mlruns"
ARTIFACT_PATH = "captcha_ocr_outputs"
SOURCE_CODE_PATH = "/kaggle/input/datasets/adritarahman3/ocr-notebook-1" 

if not os.path.exists(MLFLOW_DIR):
    os.makedirs(MLFLOW_DIR)


mlflow.set_tracking_uri(f"file://{MLFLOW_DIR}")
mlflow.set_experiment("Captcha_ResNet_OCR")


mlf_logger = MLFlowLogger(
    experiment_name="Captcha_ResNet_OCR",
    tracking_uri=f"file://{MLFLOW_DIR}"
)


model = OCRModel(num_chars=ALL_CHAR_SET_LEN, max_length=MAX_CAPTCHA)
data_module = OCRDataModule(batch_size=BATCH_SIZE)
data_module.setup()


trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    accelerator="auto",
    devices=1,
    logger=mlf_logger,
    log_every_n_steps=10
)


trainer.fit(model, data_module)
run_id = mlf_logger.run_id


with mlflow.start_run(run_id=run_id):
    
    val_results = trainer.validate(model, datamodule=data_module)
    if val_results:
        for k, v in val_results[0].items():
            mlflow.log_metric(k, v)

    
    local_weights_name = "model_weights.pth"
    torch.save(model.state_dict(), local_weights_name)
    mlflow.log_artifact(local_weights_name, artifact_path=ARTIFACT_PATH)

    
    if os.path.exists(SOURCE_CODE_PATH):
        mlflow.log_artifact(SOURCE_CODE_PATH, artifact_path="source_code")
    
   
    mlflow.pytorch.log_model(model, artifact_path="model_flavor")

    
    print(f"✅ Weights and Notebook saved in mlruns/{run_id}/artifacts")


experiment_id = mlflow.get_experiment_by_name("Captcha_ResNet_OCR").experiment_id
artifact_src = f"{MLFLOW_DIR}/{experiment_id}/{run_id}/artifacts"
zip_name = "/kaggle/working/ocr_project_results"

shutil.make_archive(zip_name, 'zip', artifact_src)

display(FileLink(zip_name + ".zip"))

/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/02/18 04:05:02 INFO mlflow.tracking.fluent: Experiment with name 'Captcha_ResNet_OCR' does not exist. Creating a new experiment.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ resnet         │ ResNet             │ 11.5 M │ train │     0 │
│ 1 │ train_accuracy │ MulticlassAccuracy │      0 │ train │     0 │
│ 2 │ val_accuracy   │ MulticlassAccuracy │      0 │ train │     0 │
└───┴────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 11.5 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 11.5 M                                                                                               
Total estimated model params size (MB): 46                                                                         
Modules in train mode: 74                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 
'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

...

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.5857142806053162     │
│         val_loss          │    1.8107324838638306     │
└───────────────────────────┴───────────────────────────┘

2026/02/18 04:06:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/18 04:06:11 WARNING mlflow.pytorch: Saving pytorch model by Pickle or CloudPickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is to set 'export_model' to True to save the pytorch model using the safe graph model format.
2026/02/18 04:06:11 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.8.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/18 04:06:30 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu1

✅ Weights and Notebook saved in mlruns/c724deacab2640f8a76495efa10fa90f/artifacts


/kaggle/working/ocr_project_results.zip

In [13]:
import shutil
from IPython.display import FileLink

shutil.make_archive('mlruns', 'zip', '/kaggle/working/mlruns')

FileLink(r'mlruns.zip')

/kaggle/working/mlruns.zip